In [1]:
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
from matplotlib.widgets import Slider
import matplotlib.patheffects as path_effects
import os
from astropy.io import fits
from glob import glob
import pandas as pd

In [63]:
class DataCube:
	"""
	An object combining multiple 1D and 2D spectra from a given folder.
	
	
	Parameters :
	----------
	folder :
		path to the folder containing the final data. Will loop on every fits file 
		and keep the one containing the _s2d or _x1d suffix
	
	Properties : 
	----------
	table : 
		a table used for making a correspondence between a source id, a s2d file and a x1d file. 
		Those files are stored as a list of paths 
	"""
	
	def __init__(self, folder):

		s2dList = [[file] for file in sorted(glob(os.path.join(folder, '*_s2d*'))) if (len(fits.open(file)[1].header.get("SHUTSTA", "")) > 1)]
		x1dList = [[x if os.path.exists(x := file[0].replace("_s2d", "_x1d")) else [None]] for file in s2dList]
		#s2dList = [[file] for file in sorted(glob(os.path.join(folder, '*_s2d*')))]
		#x1dList = [[x] if os.path.exists(x := file[0].replace("_s2d", "_x1d")) else [None] for file in s2dList]
		sourceList = [fits.open(file[0])[1].header["SOURCEID"] for file in s2dList]
		
		self.table = pd.DataFrame({"sourceID": sourceList, "s2d": s2dList, "x1d": x1dList})

		# Initializes the dataframe which will contain the data models
		self.dataTable = pd.DataFrame({"sourceID" : [], "s2d" : [], "x1d" : []})

		
		
	def combineDataCube(self, datacube,i=1):
		"""
		Combines 2 Datacubes
		Parameters
		----------
		datacube :
			Another datacube to be appended to this one, or None, which will fill append a none to every list in the array
		i : how many Nones to add. This depends on if it's the 1st call of the function or the 2nd
		"""
		
		print("Starting Combining Datacubes")

		if datacube is None:
			for i in range(len(self.table)):
				self.table["s2d"][i].append(None)
				self.table["x1d"][i].append(None)
		else:
			print("Merging...")
			# Perform an outer join to include all sourceIDs from both DataFrames
			merged = self.table.merge(
				datacube.table, on="sourceID", how="outer", suffixes=("_self", "_other")
			)

			print("Replacing empty values...")
			# Fill missing values in 's2d' and 'x1d' columns with empty lists
			merged["s2d_self"] = merged["s2d_self"].apply(lambda x: x if isinstance(x, list) else [None for _ in range(i)])
			merged["s2d_other"] = merged["s2d_other"].apply(lambda x: x if isinstance(x, list) else [None])
			merged["x1d_self"] = merged["x1d_self"].apply(lambda x: x if isinstance(x, list) else [None for _ in range(i)])
			merged["x1d_other"] = merged["x1d_other"].apply(lambda x: x if isinstance(x, list) else [None])

			print("Appending paths...")
			# Combine the 's2d' and 'x1d' columns
			merged["s2d"] = merged["s2d_self"] + merged["s2d_other"]
			merged["x1d"] = merged["x1d_self"] + merged["x1d_other"]

			# Keep only necessary columns: 'sourceID', 's2d', 'x1d'
			self.table = merged[["sourceID", "s2d", "x1d"]]
		
		print("Finished Combining Datacubes!")

	def preloadDataCube(self):
		"""
		Initializes self.dataTable, a table structurally identical to self.table, 
		except the paths are replaced by the corresponding datamodels
		"""
		print("Starting loading data...")
		print("Copying...")
		self.dataTable = self.table.copy()
		
		# Process lists of file paths
		def processList(fileList):
			return [fits.open(file) if isinstance(file, str) else None for file in fileList]
		
		print("Loading...")
		# Process the 'x1d' and 's2d' columns
		self.dataTable["x1d"] = self.dataTable["x1d"].apply(processList)
		self.dataTable["s2d"] = self.dataTable["s2d"].apply(processList)
		
		print("Finished loading data!")
		
	def exploreDataCube(self):
		fig, axes = plt.subplots(3, 1, figsize=(18, 7), gridspec_kw={'height_ratios': [1, 1, 8]})
		plt.subplots_adjust(left=0.1, bottom=0.15, right=0.9, top=0.9, hspace=0)

		idx = 0

		def drawExtraction(axe, x1d):
			x0 = x1d.header["EXTRXSTR"]
			x1 = x1d.header["EXTRXSTP"]
			y0 = x1d.header["EXTRYSTR"]
			y1 = x1d.header["EXTRYSTP"]
			axe.vlines((x0, x1), (y0, y0), (y1, y1), color='r', linestyles='dashed', linewidth=0.5)
			axe.hlines((y0, y1), (x0, x0), (x1, x1), color='r', linestyles='dashed', linewidth=0.5)

		# Update sourceID
		def update(val):
			idx = int(slider.val)  # Get the current slider value

			axes[-1].clear()  # Clear the current error bar plot

			legends = ["BNBG", "Basic Pipeline"]
			colors = ["blue", "orange"]
			for i in range(len(legends)):
				axes[i].clear()
				if not self.dataTable["s2d"][idx][i] is None:
					z1, z2 = ZScaleInterval().get_limits(self.dataTable["s2d"][idx][i][1].data)
					axes[i].imshow(self.dataTable["s2d"][idx][i][1].data, aspect='auto', vmin=z1, vmax=z2, cmap="viridis", origin="lower", interpolation="none")
					drawExtraction(axes[i], self.dataTable["x1d"][idx][i][1])

				text = axes[i].text(0.02, 0.3, legends[i], color="w", transform=axes[i].transAxes)
				text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='black'), path_effects.Normal()])

				if not self.dataTable["x1d"][idx][i] is None and legends[i] != "No Subtract":
					MJyToJy = 1
					if self.dataTable["s2d"][idx][i][1].header["BUNIT"] == "MJy/sr":
						MJyToJy = self.dataTable["s2d"][idx][i][1].header["PIXAR_SR"]
					wavelength = self.dataTable["x1d"][idx][i][1].data["WAVELENGTH"]
					flux = self.dataTable["x1d"][idx][i][1].data["FLUX"] * MJyToJy
					err = self.dataTable["x1d"][idx][i][1].data["FLUX_ERROR"] * MJyToJy
					axes[-1].plot(wavelength, flux, label=legends[i], color=colors[i])
					axes[-1].plot(wavelength, err, color=colors[i], alpha=0.5, linewidth=0.5)

			axes[-1].text(0.05, 0.05, f"SourceID: {self.dataTable['sourceID'][idx]}", color="k", transform=axes[-1].transAxes, size=15)
			axes[-1].set_xlabel(r"$\lambda$ (µm)")
			axes[-1].set_ylabel(r"Flux (Jy)")
			axes[-1].legend()
			axes[-1].grid()
			fig.canvas.draw_idle()


		def onKey(event):
			current = slider.val
			if event.key == "right":  # Move slider one step right
				new = min(current + 1, N - 1)  # Ensure within bounds
				slider.set_val(new)
			elif event.key == "left":  # Move slider one step left
				new = max(current - 1, 0)  # Ensure within bounds
				slider.set_val(new)

		# Slider
		ax_slider = plt.axes((0.2, 0.05, 0.6, 0.03))
		N = len(self.dataTable["sourceID"])
		slider = Slider(ax_slider, 'Source', 0, N - 1, valinit=idx, valstep=1)
		update(idx)

		# Attach the update function to the slider
		slider.on_changed(update)

		# Connect keypress handler
		fig.canvas.mpl_connect("key_press_event", onKey)

		# Show the plot
		plt.show()


In [64]:
dc = DataCube("/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final")
dc.combineDataCube(DataCube("/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final"))

dc.table

Starting Combining Datacubes
Merging...
Replacing empty values...
Appending paths...
Finished Combining Datacubes!


,sourceID,s2d,x1d
0,-168,"[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000168_nirspec_clear-prism_s2d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000168_nirspec_clear-prism_s2d.fits]","[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000168_nirspec_clear-prism_x1d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000168_nirspec_clear-prism_x1d.fits]"
1,-167,"[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000167_nirspec_clear-prism_s2d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000167_nirspec_clear-prism_s2d.fits]","[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000167_nirspec_clear-prism_x1d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000167_nirspec_clear-prism_x1d.fits]"
2,-166,"[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000166_nirspec_clear-prism_s2d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000166_nirspec_clear-prism_s2d.fits]","[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000166_nirspec_clear-prism_x1d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000166_nirspec_clear-prism_x1d.fits]"
3,-165,"[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000165_nirspec_clear-prism_s2d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000165_nirspec_clear-prism_s2d.fits]","[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000165_nirspec_clear-prism_x1d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000165_nirspec_clear-prism_x1d.fits]"
4,-163,"[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000163_nirspec_clear-prism_s2d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000163_nirspec_clear-prism_s2d.fits]","[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_v000000163_nirspec_clear-prism_x1d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_v000000163_nirspec_clear-prism_x1d.fits]"
...,...,...,...
303,43262,"[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o063_s000043262_nirspec_clear-prism_s2d.fits, /home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final/jw01345-o063_s000043262_nirspec_clear-prism_s2d.fits]","[/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final/jw01345-o06

In [65]:
dc.preloadDataCube()
dc.dataTable

Starting loading data...
Copying...
Loading...
Finished loading data!


,sourceID,s2d,x1d
0,-168,"[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c66a75e78d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad2b7350>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66addbf8d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66b0345190>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66b0347f10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66b0347a50>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66b0344510>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a0441c10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a0441e10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c669fd7af90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66add67c10>], [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c66a057c710>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66add66210>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66add65f50>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66acbfda10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66acbfdad0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66acbfe250>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66acbfc790>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a5e68b10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a5e682d0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a878e990>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a1a9e750>]]","[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c669a96cc90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a7acdc10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66b0381310>], [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c669844bd10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a7ace650>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a0f21390>]]"
1,-167,"[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c66afbae950>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad098350>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad09bf50>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad34f1d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad34ed10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad34c610>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad34e610>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a761cf50>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a761e010>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a493cad0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a4b58550>], [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c66aee41e10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a5dcf390>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a5dccd90>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a7188bd0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a718b790>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a718b5d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66a7189e10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad93bd10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad938dd0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66ad3ca910>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66ae6e86d0>]]","[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c669adaf390>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a0f23cd0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66a8157ed0>], [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c66983cf3d0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66ae0a1c10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7c66b021d450>]]"
2,-166,"[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7c66b022ac50>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66ad269a10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66aff80510>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66aff80150>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7c66aff81650>, <astropy.io.fits.hdu.

In [66]:
%matplotlib Qt5Agg

In [67]:
dc.exploreDataCube()